# Model Building

In order to predict whether the sensors need maintenance or not, we had python scripts that simulated data in order for us to train our model. We previously analyzed the data, and concluded that, as our data was simulated, no preprocessing was needed, because it had no typos nor outliers.

In [40]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [41]:
temperature= pd.read_csv('./data/temperature_sensor_data.csv')

pressure = pd.read_csv('./data/pressure_sensor_data.csv')

airflow = pd.read_csv('./data/airflow_sensor_data.csv')

print("Temperature data:")
print(temperature.head(5))
print("Pressure data:")
print(pressure.head(5))
print("Airflow data:")
print(airflow.head(5))

Temperature data:
             Timestamp Sensor ID  Temperature Reading (°C) Operational Status  \
0  2023-01-01 00:00:00     TS313                 21.224431               Down   
1  2023-01-01 01:00:00     TS313                 23.510171            Working   
2  2023-01-01 02:00:00     TS313                 22.493797            Working   
3  2023-01-01 03:00:00     TS313                 21.049739            Working   
4  2023-01-01 04:00:00     TS313                 21.540939  Needs Maintenance   

  Last Maintenance Date  Ambient Humidity Heat Source Proximity Sensor Model  \
0   2022-11-22 00:00:00         59.450818                Medium        TS-T2   
1   2022-08-23 01:00:00         55.940795                   Far        TS-T2   
2   2022-08-30 02:00:00         48.724091                Medium        TS-T2   
3   2022-09-20 03:00:00         68.036250                   Far        TS-T2   
4   2022-07-30 04:00:00         44.025957                   Far        TS-T2   

  Last Calibra

We realised a 80/20 split would be the best four our models.

In [42]:
def split_data(df, test_size=0.2):
    X = df.drop('Operational Status', axis=1)
    y = df['Operational Status']

    X = pd.get_dummies(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

X_train_temp, X_test_temp, y_train_temp, y_test_temp = split_data(temperature)
X_train_press, X_test_press, y_train_press, y_test_press = split_data(pressure)
X_train_airflow, X_test_airflow, y_train_airflow, y_test_airflow = split_data(airflow)

## Models

We studied different models, and ended up using RandomForestClassifier, GradientBoostingClassifier and SVC.

In [43]:
def train_and_evaluate(X_train, X_test, y_train, y_test, model, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy of {model_name}: {accuracy * 100:.2f}%")

rf_model = RandomForestClassifier(random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)
svm_model = SVC(random_state=42)

print("Temperature Sensor Data:")
train_and_evaluate(X_train_temp, X_test_temp, y_train_temp, y_test_temp, rf_model, "Random Forest")
train_and_evaluate(X_train_temp, X_test_temp, y_train_temp, y_test_temp, gb_model, "Gradient Boosting")
train_and_evaluate(X_train_temp, X_test_temp, y_train_temp, y_test_temp, svm_model, "SVM")

print("\nPressure Sensor Data:")
train_and_evaluate(X_train_press, X_test_press, y_train_press, y_test_press, rf_model, "Random Forest")
train_and_evaluate(X_train_press, X_test_press, y_train_press, y_test_press, gb_model, "Gradient Boosting")
train_and_evaluate(X_train_press, X_test_press, y_train_press, y_test_press, svm_model, "SVM")

print("\nAirflow Sensor Data:")
train_and_evaluate(X_train_airflow, X_test_airflow, y_train_airflow, y_test_airflow, rf_model, "Random Forest")
train_and_evaluate(X_train_airflow, X_test_airflow, y_train_airflow, y_test_airflow, gb_model, "Gradient Boosting")
train_and_evaluate(X_train_airflow, X_test_airflow, y_train_airflow, y_test_airflow, svm_model, "SVM")

Temperature Sensor Data:
Accuracy of Random Forest: 80.00%
Accuracy of Gradient Boosting: 80.00%
Accuracy of SVM: 80.00%

Pressure Sensor Data:
Accuracy of Random Forest: 88.50%
Accuracy of Gradient Boosting: 85.50%
Accuracy of SVM: 88.50%

Airflow Sensor Data:
Accuracy of Random Forest: 79.50%
Accuracy of Gradient Boosting: 79.50%
Accuracy of SVM: 79.50%


We tried to improve our models using hyperparameter tuning, but didn't find meaningful improvements.

In [44]:
def hyperparameter_tuning(model, param_grid, X_train, y_train, X_test, y_test):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                               cv=3, n_jobs=-1, verbose=2, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    print("\nBest parameters:", best_params)

    model.set_params(**best_params)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy with best parameters: {accuracy * 100:.2f}%\n")

    return model, accuracy

rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt'],  
    'max_depth': [10, 20, 30, None],   
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],     
    'bootstrap': [True, False]    
}

gb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

svm_param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']
}

models = [rf_model, gb_model, svm_model] 
datasets = [(X_train_temp, X_test_temp, y_train_temp, y_test_temp), 
            (X_train_press, X_test_press, y_train_press, y_test_press), 
            (X_train_airflow, X_test_airflow, y_train_airflow, y_test_airflow)]

for model in models:
    for dataset in datasets:
        X_train, X_test, y_train, y_test = dataset
        if isinstance(model, RandomForestClassifier):
            hyperparameter_tuning(model, rf_param_grid, X_train, y_train, X_test, y_test)
        elif isinstance(model, GradientBoostingClassifier):
            hyperparameter_tuning(model, gb_param_grid, X_train, y_train, X_test, y_test)
        elif isinstance(model, SVC):
            hyperparameter_tuning(model, svm_param_grid, X_train, y_train, X_test, y_test)

Fitting 3 folds for each of 432 candidates, totalling 1296 fits


[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_featu

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
648 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
648 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/site-packages/sklearn/uti


Best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy with best parameters: 80.00%

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
648 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
648 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/site-packages/sklearn/uti


Best parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy with best parameters: 88.50%

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_

/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
648 fits failed out of a total of 1296.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
648 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/site-packages/sklearn/base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/site-packages/sklearn/uti


Best parameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Accuracy with best parameters: 79.50%

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.3s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   7.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  14.4s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  14.8s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  14.8s
[CV] END learning_rate=0.01, m

KeyboardInterrupt: 